In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
import copy
sys.path.append('plotting_utils')
sys.path.append('simulation')
from plotting_utils.plotting import *
from simulation.run_multiple_sim import SimArgs

In [ ]:
# Collection data from pkl file
file_dir = "new_data"
fname = "size_error_expt.pkl"
file_path = os.path.join(file_dir, fname)

with open(file_path,'rb') as fp:
    results = pickle.load(fp)

In [ ]:
raw_data = {}

# create empty dictionary
for policy in ['AFS', 'FIFO', 'MCS']:
    raw_data[policy] = {}
    for p_error in [0,5,10,20,30,40]:
        raw_data[policy][p_error] = list()

# fill dictionary with result key data
for k, v in results.items():
    file_path = os.path.join(file_dir, k.output_file)
    raw_data[k.scheduling_policy][int(k.p_error)].append(pd.read_csv(file_path))

# compute statistics
min_avg_jct = {}
for policy in ['AFS', 'FIFO', 'MCS']:
    for p_error in [0,5,10,20,30,40]:

        min_avg_jct.setdefault(p_error, float('inf'))
                
        c_df = pd.concat(raw_data[policy][p_error])
        c_df.reset_index(drop=True, inplace=True)

        jct = c_df['end_time'] - c_df['submit_time']
        f_c_df = c_df[c_df['estimated_start_time'] != -1]
        
        pred_jct = f_c_df['estimated_end_time'] - f_c_df['submit_time']
        actual_jct = f_c_df['end_time'] - f_c_df['submit_time']
        error = 100.0 * (pred_jct - actual_jct).abs() / pred_jct
        
        min_avg_jct[p_error] = min(min_avg_jct[p_error], jct.mean())
        raw_data[policy][p_error] = {
            'avg JCT': jct.mean(),
            'p90 JCT': jct.quantile(0.90),
            'p95 JCT': jct.quantile(0.95),
            'p99 JCT': jct.quantile(0.99),
            'avg pred error': error.mean(),
            'p90 pred error': error.quantile(0.90),
            'p95 pred error': error.quantile(0.95),
            'p99 pred error': error.quantile(0.99),
        }

for policy in ['AFS', 'FIFO', 'MCS']:
    for p_error in [0,5,10,20,30,40]:
        raw_data[policy][p_error]['avg JCT'] /= min_avg_jct[p_error]
        # raw_data[policy][p_error]['avg JCT']
size_error_sensitivity = {'PCS' if k == 'MCS' else k : v for k,v in raw_data.items()}

In [ ]:
# figure 10a
ax = plot_fig10a(size_error_sensitivity)
ax.figure

In [ ]:
# figure 10b
ax = plot_fig10b(size_error_sensitivity)
ax.set_ylim([0.95,1.3])
# ax.set_ylim([75000,100000])
ax.figure